In [1]:
import os

import yaml
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet

from utils import train_fn, get_loaders, load_checkpoint, save_checkpoint, check_performance, plot_frames
from models import ResNet18_RNN

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

EPOCHS = config["training_parameters"]["epochs"]
LEARNING_RATE = config["training_parameters"]["learning_rate"]

TRAIN_CSV = config["dataloader_parameters"]["train_csv"]
VAL_CSV = config["dataloader_parameters"]["val_csv"]
ROOT_DIR = config["dataloader_parameters"]["root_dir"]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2
MAX_FRAMES = config["dataloader_parameters"]["max_frames"]
NUM_WORKERS = config["dataloader_parameters"]["num_workers"]
PIN_MEMORY = config["dataloader_parameters"]["pin_memory"]

RNN_TYPE = config["model_parameters"]["rnn_type"]
RNN_LAYERS = config["model_parameters"]["rnn_layers"]
RNN_HIDDEN = config["model_parameters"]["rnn_hidden"]
TRAINABLE_BACKBONE = config["model_parameters"]["trainable_backbone"]
HEADS_PARAMS = config["model_parameters"]["heads_params"]
HEADS_PARAMS["in_features"] = RNN_HIDDEN * MAX_FRAMES

In [2]:
a = {"a": 1, "b": 2}
c = {"c": 1, "d": 2}
b = {**a, **c}

In [3]:
b

{'a': 1, 'b': 2, 'c': 1, 'd': 2}

In [4]:
model = ResNet18_RNN(
    RNN_TYPE, 
    RNN_LAYERS, 
    RNN_HIDDEN, 
    HEADS_PARAMS, 
    TRAINABLE_BACKBONE
)

In [5]:
resnet_transforms = resnet.ResNet18_Weights.DEFAULT.transforms()
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224)
    # resnet_transforms,
])
val_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    # resnet_transforms,
])

In [15]:
dummy_data = torch.rand(69, 3, 512, 512)
dummy_data = train_transforms(dummy_data).unsqueeze(0)

In [16]:
dummy_data.shape

torch.Size([1, 69, 3, 224, 224])

In [14]:
model(dummy_data.unsqueeze(0))

{'body_rotation_type': tensor([[-0.2465,  0.2159, -0.1809]], grad_fn=<AddmmBackward0>),
 'body_rotation_number': tensor([[0.1713]], grad_fn=<AddmmBackward0>),
 'board_rotation_type': tensor([[ 0.0505,  0.2424, -0.0416]], grad_fn=<AddmmBackward0>),
 'board_rotation_number': tensor([[-0.1312]], grad_fn=<AddmmBackward0>),
 'flip_type': tensor([[-0.0651, -0.0940,  0.2286]], grad_fn=<AddmmBackward0>),
 'flip_number': tensor([[0.1031]], grad_fn=<AddmmBackward0>),
 'landed': tensor([[-0.2385,  0.2354]], grad_fn=<AddmmBackward0>),
 'stance': tensor([[-0.0009,  0.0075, -0.0666, -0.0916]], grad_fn=<AddmmBackward0>)}

In [9]:
import pandas as pd
pd.read_csv(TRAIN_CSV).shape

(220, 14)

In [10]:
n = "fakie double kickflip"
names = [f"{n} 1", f"{n} 2", f"{n} 4"]

In [16]:
int(names[0].split(n)[-1].strip())

1

In [17]:
26 * 50

1300

In [4]:
train_loader, val_loader = get_loaders(
    TRAIN_CSV,
    VAL_CSV,
    ROOT_DIR,
    MAX_FRAMES,
    BATCH_SIZE,
    train_transforms,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY
)

In [5]:
len(train_loader)

110

In [6]:
len(val_loader)

28

In [6]:
loss_fns = {
    'body_rotation_type': nn.CrossEntropyLoss(),
    'body_rotation_number': nn.MSELoss(),
    'board_rotation_type': nn.CrossEntropyLoss(),
    'board_rotation_number': nn.MSELoss(),
    'flip_type': nn.CrossEntropyLoss(),
    'flip_number': nn.MSELoss(),
    'landed': nn.CrossEntropyLoss(),
    'stance': nn.CrossEntropyLoss()
}

In [7]:
l = check_performance(val_loader, model, loss_fns, DEVICE)

c:\Users\Eduardo Pacheco\.virtualenvs\SkateAI-K9OkN_Dw\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [78]:
for data, target in train_loader:
    break

In [7]:
target = {key: target[key].float() if 'number' in key else target[key] for key in target.keys()}

In [106]:
len(train_loader)

86

In [83]:
from models.utils import Heads

In [84]:
m = Heads(**HEADS_PARAMS)

In [86]:
b = [1,2,3]
c = [4,5,6]
d = [a for a in zip(b,c)]
d

[(1, 4), (2, 5), (3, 6)]

In [90]:
import yaml

In [91]:
with open("config.yaml", 'r') as f:
    config = yaml.safe_load(f)

In [99]:
for (neuron, dropout) in config["model_parameters"]["heads_params"]["byrt"]:
    print(neuron, dropout)

256 0.5
128 0.2
16 0.3


In [100]:
def my_func(a=10):
    return a+1

In [8]:
for val in target.values():
    print(val.dtype)

torch.int64
torch.float32
torch.int64
torch.float32
torch.int64
torch.float32
torch.int64
torch.int64


In [25]:
model.eval();
with torch.no_grad():
    predictions = model(data)

In [19]:
import torch.nn.functional as F

In [34]:
loss_dict = dict()
loss_total = 0.0
for key in predictions.keys():
    loss_dict[key] = loss_fns[key](predictions[key], target[key])
    loss_total += loss_dict[key]

c:\Users\Eduardo Pacheco\.virtualenvs\SkateAI-K9OkN_Dw\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [35]:
loss_dict

{'body_rotation_type': tensor(1.0457),
 'body_rotation_number': tensor(0.2678),
 'board_rotation_type': tensor(0.9940),
 'board_rotation_number': tensor(0.1582),
 'flip_type': tensor(0.7797),
 'flip_number': tensor(0.0787),
 'landed': tensor(0.1994),
 'stance': tensor(1.2671)}

# Testing wandb

In [9]:
os.environ["WANDB_DATASET_ARTIFACT"]

'batb_raw'

In [11]:
api = wandb.Api()
artifact = api.artifact(f"{os.environ['WANDB_PROJECT']}/{os.environ['WANDB_DATASET_ARTIFACT']}:latest")

In [38]:
artifact["eda_table"]

wandb: Downloading large artifact batb_raw:latest, 62.49MB. 450 files... 
wandb:   450 of 450 files downloaded.  
Done. 0:0:1.4


In [12]:
from pathlib import Path

In [31]:
p = Path(artifact.download())

wandb: Downloading large artifact batb_raw:latest, 62.49MB. 450 files... 
wandb:   450 of 450 files downloaded.  
Done. 0:0:3.3


In [33]:
import shutil

In [39]:
shutil.rmtree(p.parent)